# Exploratory Data Analysis
### Daten Laden
### Univariate Analyse
### Bivariate Analyse
### Multivariate Analyse

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout = True, dpi = 110)
plt.rc('axes', labelweight = 'bold', titlesize = 14, titleweight = 'bold')
plt.rc('font', size = 11)

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/used-car-prediction-notebook-01/cleaned_data_01.csv")
print(df.shape)
df.head()

## Univariate Analyse
Hier untersuchen wir:
- Die Verteilung unserer numerischen Features
- Die Verteilung der kategorischen Features
- Offensichtliche Außreiser?
- Die Kardinalitäten unserer kategorischen Features

In [ ]:
df.info()

In [ ]:
df.describe()

ein erster Blick auf unsere numerischen Features legt keine offensichtlichen Ausreiser nahe. Ein maximaler Kilometerstand von 1.000.000 scheint plausibel ebenso wie ein ein maximaler Verkaufspreis von knapp 400.000, maximale Anzahl von Besitzern von 6 usw.

In [ ]:
#Filtern von kontinuierlichen Features
cont_nums = ['sale_price', 'kms_run', 'original_price']

#Plotten wir die Verteilung unserer kontinuierlichen Features mit Histogrammen
plt.figure(figsize = (12,3))
for index, col in enumerate(cont_nums):
    plt.subplot(1, 3, index+1)
    sns.histplot(df[col])

Wie zu erwarten sind unsere kontinuierlichen Features rechtsschief. Das Feature original_price sieht von der Verteilung her verdächtig ähnlich aus wie sale_price.

In [ ]:
cat_disc_cols = [col for col in df.columns if col not in cont_nums]
high_kard_cols = []
low_kard_cols = []
for col in cat_disc_cols:
    if df[col].nunique() > 20:
        high_kard_cols.append(col)
    else:
        low_kard_cols.append(col)
print(high_kard_cols)
print(len(low_kard_cols))

In [ ]:
plt.figure(figsize = (4,3))
sns.kdeplot(df['age'])

In [ ]:
plt.figure(figsize = (12,9))
for index, col in enumerate(low_kard_cols):
    plt.subplot(3, 3, index +1)
    order = df[col].value_counts().index
    sns.countplot(x= df[col], color = '#1f77b4', order = order)
    plt.title(col, fontsize = 12)
    plt.xticks(rotation = 45, ha = 'right')
    plt.xlabel('')

### Analyse der kategorialen Features

- Die meisten Fahrzeuge verwenden Petrol als Kraftstoff, gefolgt von Diesel. Alternative Antriebe wie CNG oder Electric kommen nur selten vor.

- Hatchbacks und Sedans sind die häufigsten Fahrzeugtypen, größere oder teurere Modelle wie SUVs sind deutlich seltener.

- Manuelle Getriebe dominieren gegenüber Automatikfahrzeugen.

- Viele Fahrzeuge stammen aus Städten wie Mumbai, New Delhi und Bengaluru.

- Die meisten Autos hatten nur einen Vorbesitzer und verfügen über ein gültiges Fitness Certificate, während Garantien selten vorhanden sind.

- Die Bewertungen sind stark unausgeglichen und konzentrieren sich auf positive Einschätzungen wie "great" oder "good".

In [ ]:
high_kard_cols.remove('age')

plt.figure(figsize = (12,6))
for index, col in enumerate(high_kard_cols):
    plt.subplot(2, 3, index + 1)
    high_freq = df[col].value_counts().nlargest(10)
    sns.barplot(x=high_freq.index, y=high_freq.values, color = '#1f77b4')
    plt.xticks(rotation=45, ha='right')

### Analyse der kategorialen Features mit hoher Kardinalität

Die Features make, model, variant, registered_city und rto zeigen deutliche Häufungen in den größten zehn Kategorien. Besonders make und variant sind stark dominiert von wenigen Werten (z. B. „Maruti“ und „VXI“), während andere Kategorien nur selten vorkommen.
Registered_city und rto sind etwas gleichmäßiger verteilt, zeigen aber ebenfalls eine Konzentration auf wenige große Städte bzw. Regionen.
Das Feature car_name stellt im Wesentlichen eine Kombination aus make und model dar, weshalb es keine zusätzliche Information liefert und redundant ist.

## Bivariate Analyse

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def get_mutual_info(data, target):
    X = data.copy().drop(columns = 'original_price')
    y = X.pop('sale_price')

    #Kategorische Features in ein numerisches Format bringen:
    for col in X.select_dtypes('object'):
        X[col], _ = X[col].factorize()

    mi_scores = mutual_info_regression(X, y, discrete_features = True)
    mi_scores = pd.Series(mi_scores, name = 'MI-Scores', index = X.columns)
    mi_scores = mi_scores.sort_values(ascending = False)

    print(mi_scores)
    
    if 'original_price' in data.columns:
        X2 = data[['original_price', 'sale_price']].copy()
        X2 = X2.dropna()
        y2 = X2.pop('sale_price')
        mi_score = mutual_info_regression(X2, y2)
        mi_score = pd.Series(mi_score, name = 'MI-Original_Price', index = X2.columns)
        print(mi_score)

In [ ]:
get_mutual_info(df, 'sale_price')

Variant, car_name, model zeigen in der ersten Aufzählung den größten Zusammenhang mit unserem Target. Dies ist erstmal sehr intuitiv zu verstehen, da man für einen Ferrari natürlich einen höheren Preis erwarten würde als für einen Dacia. Auffällig hoch ist hier wieder original_price. Da bei dem Datensatz keine Informationen über die Erhebung der Daten vorhanden war, lässt sich schwer mit Gewissheit sagen, woran das liegt. Eine Möglichkeit, wäre, dass es sich bei dem Feature original_price um ein künstlich erzeugtes feature abgeleitet von sale_price handelt. Mal sehen ob uns ein Scatterplot mehr verrät.

In [ ]:
sns.pairplot(df[['sale_price', 'original_price', 'kms_run', 'age', 'total_owners']], corner = True, diag_kind = 'kde')

Der Pairplot bestätigt meine Vermutung. Der Zusammenhang zwischen original_price und sale_price ist zu linear um natürlichen Ursprungs zu sein.
Die Verwendung des Features würde vermutlich zu sehr genauen Vorhersagen unseres Modells führen. In der Realität wird der Ursprüngliche Preis häufig bekannt sein und er sollte auch ein guter Indikator für den Wiederverkaufswert sein. Allerdings sollten die anderen Eigenschaften wie Alter, Zustand etc höhere Schwankungen im Zusammenhang zwischen original_price und sale_price verursachen. Aus diesem Grund werde ich bei meinem weiteren Verfahren original_price zuerst weglassen und später möglicherweise ein weiters Modell trainieren, dass original_price verwendet.

In [ ]:
original_price = df.pop('original_price')

Untersuchen wir nun den Zusammenhang der Kategorischen Features um zu sehen, welche wir davon wirklich benötigen. Eine schnelle und performante Methode ist Cramer's V.

In [ ]:
from scipy.stats import chi2_contingency

def cramers_v(X, y):
    confusion_matrix = pd.crosstab(X, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r, k = confusion_matrix.shape

    phi2corr = max(0, phi2 - ((k - 1)*(r - 1))/(n - 1))
    rcorr = r - ((r - 1)**2)/(n - 1)
    kcorr = k - ((k - 1)**2)/(n - 1)

    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

In [ ]:
cats = df.select_dtypes(['object', 'bool']).columns
cramers = pd.DataFrame(index=cats, columns=cats)

for col1 in cats:
    for col2 in cats:
        cramers.loc[col1, col2] = cramers_v(df[col1], df[col2])

cramers = cramers.astype(float)
plt.figure(figsize = (8,8))
mask = abs(cramers < 0.5)
sns.heatmap(cramers, annot=True, cmap="coolwarm", center=0.5, mask = mask)
plt.title("Cramér’s V – Zusammenhang kategorialer Features")
plt.show()

In [ ]:
for col in df.select_dtypes('object'):
    print(f'Anzahl Kategorien in {col}: {df[col].nunique()}')

- Hier lässt sich sehr schön erkennnen, dass carname einen perfekten Zusammenhang mit make und model hat. Dies ist logisch, da car_name nur die Aneinanderreihung der beiden Features ist. Wir können car_name also einfach droppen. Damit haben wir ein Feature mit hoher Kardinalität weniger, dass uns Sorgen bereitet.

- Variant besitzt ebenfalls sehr viele verschiedene Kategorien und ist zu einem großen Teil redundant mit anderen Features, wie zu Beispiel fuel_type. Ich werde versuchen es so zu zerlegen, dass nützliche Informationen weitergetragen werden.

- rto ist eine Kombination aus registerd_state und einer Nummer für eine weitere unterteilung. Diesen zu zerlegen in rto_state und rto_code macht registered state überflüssig und könnte möglicherweise registered_city überflüssig machen

- registered_city und city hängen eng miteinander zusammen, allerdings nicht so eng wie erwartet. Grund dafür dürfte unterschiedliche Bennenung und Kategorisierung sein

In [ ]:
df.drop(columns = 'car_name', inplace = True)

In [ ]:
df.to_csv("/kaggle/working/cleaned_data_02.csv", index=False)